# Chapter 4 - Programming Exercises

### [Link](https://runestone.academy/runestone/books/published/pythonds3/BasicDS/Exercises.html)

### 1. Modify the infix-to-postfix algorithm so that it can handle errors.

We have to see what are errors in the context of this algorithm. It turns out this list can present some common errors:
* The number of open parentheses must equal the number of closed parentheses.
* The token is something else than the ones accepted for the function (`"ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789()+-/*"`).

In [11]:
from pythonds3.basic import Stack

def infix_to_postfix(infix_expr):
    if infix_expr.count("(") != infix_expr.count(")"):
        print(f"Error! Closed parentheses (n = {infix_expr.count('(')}) don't match open parentheses (n = {infix_expr.count(')')})!")
        return 
    
    prec = {}
    prec["*"] = 3
    prec["/"] = 3
    prec["+"] = 2
    prec["-"] = 2
    prec["("] = 1
    op_stack = Stack()
    postfix_list = []
    token_list = infix_expr.split()

    for token in token_list:
        if token not in "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789()+-/*":
            print(f"Error, token {token} not recognized!")
        elif token in "ABCDEFGHIJKLMNOPQRSTUVWXYZ" or token in "0123456789":
            postfix_list.append(token)
        elif token == "(":
            op_stack.push(token)
        elif token == ")":
            top_token = op_stack.pop()
            while top_token != "(":
                postfix_list.append(top_token)
                top_token = op_stack.pop()
        else:
            while (not op_stack.is_empty()) and (prec[op_stack.peek()] >= prec[token]):
                postfix_list.append(op_stack.pop())
            op_stack.push(token)

    while not op_stack.is_empty():
        postfix_list.append(op_stack.pop())

    return " ".join(postfix_list)

### 2. Modify the postfix evaluation algorithm so that it can handle errors.

I can see that we have to add an exception when we are diving by zero. I decided to still return infinity.

In [12]:
import math

def postfix_eval(postfix_expr):
    operand_stack = Stack()
    token_list = postfix_expr.split()

    for token in token_list:
        if token in "0123456789":
            operand_stack.push(int(token))
        else:
            operand2 = operand_stack.pop()
            operand1 = operand_stack.pop()
            result = do_math(token, operand1, operand2)
            operand_stack.push(result)
    return operand_stack.pop()


def do_math(op, op1, op2):
    if op == "*":
        return op1 * op2
    elif op == "/":
        try:
            result = op1 / op2
        except ZeroDivisionError:
            print(f"Division by 0 will give an infinite number!")
            result = op1 * math.inf
        finally:
            return result
    elif op == "+":
        return op1 + op2
    else:
        return op1 - op2

### 3. Implement a direct infix evaluator that combines the functionality of infix-to-postfix conversion and the postfix evaluation algorithm. Your evaluator should process infix tokens from left to right and use two stacks, one for operators and one for operands, to perform the evaluation.

Both the modules already defined can be chained to produce a direct infix evaluator, but since we are operating with numbers, we have to modify it so that only numbers are accepted. As bonus point, I will add, as asked during the chapter, the ** (power) operator.

In [13]:
def direct_infix_evaluator(infix_expr):
    if infix_expr.count("(") != infix_expr.count(")"):
        print(f"Error! Closed parentheses (n = {infix_expr.count('(')}) don't match open parentheses (n = {infix_expr.count(')')})!")
        return 
    
    prec = {}
    prec["*"] = 4
    prec["/"] = 4
    prec["+"] = 3
    prec["-"] = 3
    prec["^"] = 2
    prec["("] = 1
    op_stack = Stack()
    operand_stack = Stack()
    postfix_list = []
    token_list = infix_expr.replace("**", "^").split()

    for token in token_list:
        if token not in "0123456789()+-/*^":
            print(f"Error, token {token} not recognized!")
        elif token in "ABCDEFGHIJKLMNOPQRSTUVWXYZ" or token in "0123456789":
            postfix_list.append(token)
        elif token == "(":
            op_stack.push(token)
        elif token == ")":
            top_token = op_stack.pop()
            while top_token != "(":
                postfix_list.append(top_token)
                top_token = op_stack.pop()
        else:
            while (not op_stack.is_empty()) and (prec[op_stack.peek()] >= prec[token]):
                postfix_list.append(op_stack.pop())
            op_stack.push(token)

    while not op_stack.is_empty():
        postfix_list.append(op_stack.pop())

    postfix_expr = " ".join(postfix_list)
    token_list = postfix_expr.split()

    for token in token_list:
        if token in "0123456789":
            operand_stack.push(int(token))
        else:
            operand2 = operand_stack.pop()
            operand1 = operand_stack.pop()
            result = do_math(token, operand1, operand2)
            operand_stack.push(result)
            
    return operand_stack.pop()


Let's now run it with different expressions.

In [14]:
print(direct_infix_evaluator("5 * 3 ** ( 4 - 2 )"))
print(direct_infix_evaluator("( ( 3 + 2 ) * ( 6 * ( 1 - 5 ** 3 ) ) )"))

13
-210


It seems it's working!